# In memory return

In some particular cases, QLM QPUs are able to return the final sampling data in the form of a numpy array (usually via some in-memory transfer).

This provide a large advantage if you want to extract data using vectorized operation over the final vector.

This notebooks uses the LinAlg qpu to illustrate this feature.

In [ ]:
import numpy as np
import time
from qat.qpus import LinAlg
from qat.lang.AQASM import Program, H, CNOT
program = Program()
for qb in program.qalloc(10):
    H(qb)
circuit = program.to_circ()

Lets start with a job requiring all the qubits to be sampled. Since LinAlg is a perfect simulator, it will try and return the full state vector as a complex numpy array.

This array can be found in the `.statevector` field of the result object.

In [ ]:
qpu = LinAlg()
result = qpu.submit(circuit.to_job())
if result.has_statevector:
    print("The result has a numpy data of shape:", result.statevector.shape)
    print("The norm of the state vector is", np.linalg.norm(result.statevector))

If we now ask LinAlg to provide a probability distribution on a subset of qubits, the resulting array will be of float type.

In [ ]:
qpu = LinAlg()
result = qpu.submit(circuit.to_job(qubits=[0, 1]))
if result.has_statevector:
    print("The result has a numpy data of shape:", result.statevector.shape)
    print(result.statevector)

LinAlg also has the capability of returning state vectors and probabiluty distributions in sparse format (a c++ map).
This c++ map is published as a python object and behaves like a standard dictionary.

This can be activated via the `sparse` keyword of LinAlg's constructor:

In [ ]:
qpu = LinAlg(sparse=True)
result = qpu.submit(circuit.to_job(qubits=[0, 1]))
if result.has_statevector:
    print("The state vector is a", type(result.statevector))
    print(result.statevector)
    for key, value in result.statevector.items():
        print(key, ":", value)

In all cases, the `Result` object will always implement some iterator returning `Sample` objects, regardless of the underlying data structure describing the state vector/result. Thus, the standard way (which might be less efficient, depending on your application) is to use a for loop:

In [ ]:
qpu = LinAlg(sparse=True)
result = qpu.submit(circuit.to_job(qubits=[0, 1, 2, 3]))
print(">>>>  Sparse mode:")
for sample in result:
    print(sample)
qpu = LinAlg(sparse=False)
print(">>>>  Full mode:")
result = qpu.submit(circuit.to_job(qubits=[0, 1, 2, 3]))
for sample in result:
    print(sample)